In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [2]:
# all the data is from NHATS
# read round 1, 2, 5, 6 sp data and sp status data
r1_status = pd.read_stata('NHATS_Round_1_Tracker_File.dta')
r1_data = pd.read_stata('NHATS_Round_1_SP_File.dta')
r1_met = pd.read_stata('NHATS_Round_1_MetNonMet.dta')

r2_status = pd.read_stata('NHATS_Round_2_Tracker_File_v2.dta')

r5_status = pd.read_stata('NHATS_Round_5_Tracker_File_V3.dta')
r5_data = pd.read_stata('NHATS_Round_5_SP_File_V2.dta')
r5_met = pd.read_stata('NHATS_Round_5_MetNonMet.dta')

r6_status = pd.read_stata('NHATS_Round_6_Tracker_File_V3.dta')

# read round 1, 2, 5, 6 cognitive status data
r1_cog = pd.read_csv('NHATS_cognition_r1.csv')
r2_cog = pd.read_csv('NHATS_cognition_r2.csv')
r5_cog = pd.read_csv('NHATS_cognition_r5.csv')
r6_cog = pd.read_csv('NHATS_cognition_r6.csv')

# read round 1, 2, 5, 6 op data
r1_op = pd.read_stata('NHATS_Round_1_OP_File_v2.dta')
r5_op = pd.read_stata('NHATS_Round_5_OP_File_V2.dta')

In [3]:
# merge round 1 data and round 1 status
r1_data = pd.merge(r1_data, r1_status[['spid', 'r1status']], on = ['spid'], how = 'left')

# merge round 1 data and round 1 cognitive variables
r1_data = pd.merge(r1_data, r1_cog[['spid', 'demclas', 'clock_scorer', 'wordrecall0_20', 'date_prvp']], 
                   on = ['spid'], how = 'left')

# merge round 1 data and round 2 status
r1_data = pd.merge(r1_data, r2_status[['spid', 'r2status']], on = ['spid'], how = 'left')

# rename round 2 cognitive variables
r2_cog.rename(columns={"demclas": "demclas_t1", "clock_scorer": "clock_scorer_t1",
                       "wordrecall0_20": "wordrecall0_20_t1", "date_prvp": "date_prvp_t1"}, inplace=True)

# merge round 1 data and round 2 cognitive variables
r1_data = pd.merge(r1_data, r2_cog[['spid', 'demclas_t1', 'clock_scorer_t1', 'wordrecall0_20_t1', 'date_prvp_t1']], 
                   on = ['spid'], how = 'left')

# subsample of persons who did not have dementia in round 1 
r1_data = r1_data[r1_data['demclas']==3]

In [4]:
# drop persons who had missing value in cognitive tests scores (value < 0)
cog_cols = ['clock_scorer', 'wordrecall0_20', 'date_prvp', 'clock_scorer_t1', 'wordrecall0_20_t1', 'date_prvp_t1']
r1_data = r1_data[~(r1_data.loc[:, cog_cols] < 0).any(axis=1)]

In [5]:
# NHATS ognitive tests identified 3 domains of cognitive functioning: memory(wordrecall0_20), orientation(date_prvp),
# and executive functioning(clock_scorer)
# these scores have different ranges: wordrecall0_20 [0,20], date_prvp [0,8], clock_scorer [0, 5]
# standardize the three scores to the same range of [0, 10]
from sklearn.preprocessing import MinMaxScaler
cols_scale = ['clock_scorer', 'wordrecall0_20', 'date_prvp',
              'clock_scorer_t1', 'wordrecall0_20_t1', 'date_prvp_t1']
scaled_values = MinMaxScaler().fit_transform(r1_data.loc[:, cols_scale])
df_scaled = pd.DataFrame(scaled_values, columns=cols_scale)
df_scaled = df_scaled * 10
r1_data.loc[:, cols_scale] = df_scaled

# create a cognitive score as the sum of the above mentioned 3 scores
r1_data['cog_score'] = r1_data[['clock_scorer', 'wordrecall0_20', 'date_prvp']].sum(axis = 1)
r1_data['cog_score_t1'] = r1_data[['clock_scorer_t1', 'wordrecall0_20_t1', 'date_prvp_t1']].sum(axis = 1)

In [6]:
# merge round 5 data and round 5 status
r5_data = pd.merge(r5_data, r5_status[['spid', 'r5status']], on = ['spid'], how = 'left')

# merge round 5 data and round 5 cognitive variables
r5_data = pd.merge(r5_data, r5_cog[['spid', 'demclas', 'clock_scorer', 'wordrecall0_20', 'date_prvp']], 
                   on = ['spid'], how = 'left')

# merge round 5 data and round 6 status
r5_data = pd.merge(r5_data, r6_status[['spid', 'r6status']], on = ['spid'], how = 'left')

# rename round 6 cognitive variables
r6_cog.rename(columns={"demclas": "demclas_t1", "clock_scorer": "clock_scorer_t1",
                       "wordrecall0_20": "wordrecall0_20_t1", "date_prvp": "date_prvp_t1"}, inplace=True)

# merge round 5 data and round 6 cognitive variables
r5_data = pd.merge(r5_data, r6_cog[['spid', 'demclas_t1', 'clock_scorer_t1', 'wordrecall0_20_t1', 'date_prvp_t1']], 
                   on = ['spid'], how = 'left')

# subsample of persons who did not have dementia in round 5 
r5_data = r5_data[r5_data['demclas']==3]

In [7]:
# drop persons who had missing value in cognitive tests scores (value < 0)
cog_cols = ['clock_scorer', 'wordrecall0_20', 'date_prvp', 'clock_scorer_t1', 'wordrecall0_20_t1', 'date_prvp_t1']
r5_data = r5_data[~(r5_data.loc[:, cog_cols] < 0).any(axis=1)]

In [8]:
# NHATS ognitive tests identified 3 domains of cognitive functioning: memory(wordrecall0_20), orientation(date_prvp),
# and executive functioning(clock_scorer)
# these scores have different ranges: wordrecall0_20 [0,20], date_prvp [0,8], clock_scorer [0, 5]
# standardize the three scores to the same range of [0, 10]
cols_scale = ['clock_scorer', 'wordrecall0_20', 'date_prvp',
              'clock_scorer_t1', 'wordrecall0_20_t1', 'date_prvp_t1']
scaled_values = MinMaxScaler().fit_transform(r5_data.loc[:, cols_scale])
df_scaled = pd.DataFrame(scaled_values, columns=cols_scale)
df_scaled = df_scaled * 10
r5_data.loc[:, cols_scale] = df_scaled

# create a cognitive score as the sum of the above mentioned 3 scores
r5_data['cog_score'] = r5_data[['clock_scorer', 'wordrecall0_20', 'date_prvp']].sum(axis = 1)
r5_data['cog_score_t1'] = r5_data[['clock_scorer_t1', 'wordrecall0_20_t1', 'date_prvp_t1']].sum(axis = 1)

# construct a sample of round 1 sp

In [9]:
# lists of relevant variables
varlist_sp = ['spid', 'r1dresid', 'is1resptype', 'r1dgender', 'r1d2intvrage', 'cog_score', 'cog_score_t1',
              'hc1health', 'hc1disescn3', 'hc1disescn8', 'hc1hosptstay', 'hc1hosovrnht', 'hc1aslep30mn', 'hc1sleepmed',
              'ht1retiresen',
              'hh1martlstat', 'hh1d2spouage', 'hh1spouseduc', 'hh1spoupchlp', 'hh1livwthspo', 'hh1dlvngarrg', 'hh1dhshldchd',
              'hh1dlvngarrg',
              'cs1dnumchild', 'cs1dnumdaugh', 'sd1smokedreg', 'sd1smokesnow', 
              'el1hlthchild', 'el1fingrowup', 'el1higstschl', 'rl1spkothlan', 'ip1nginsnurs',
              'hp1ownrentot', 'hp1mrtpadoff', 'ia1totinc', 'ia1toincimf', 'ia1toincim1']
varlist_help = ['mo1douthelp', 'mo1dinsdhelp', 'mo1dbedhelp', 'dm1helpmobil', 'ha1moneyhlp', 'sc1eathlp',
                'sc1bathhlp', 'sc1toilhlp', 'sc1dreshlp', 'mc1medstrk', 'mc1howpkupm3']
varlist_op = ['spid', 'opid', 'op1gender', 'op1relatnshp', 'op1leveledu', 'op1childinhh', 'op1martlstat', 'op1numchldrn',
              'op1numchdu18',
              'op1ishelper', 'op1helpsched',
              'op1numdayswk', 'op1numdaysmn', 'op1numhrsday', 'op1paidhelpr', 'op1dhrsmth',
              'op1outhlp', 'op1insdhlp', 'op1bedhlp', 'op1launhlp', 'op1shophlp', 'op1mealhlp', 'op1bankhlp', 'op1moneyhlp',
              'op1eathlp', 'op1bathhlp', 'op1toilhlp', 'op1dreshlp', 'op1medshlp', 'op1dochlp', 'op1insurhlp']

In [10]:
r1_sp = r1_data.loc[:, varlist_sp].copy()
r1_op_new = r1_op.loc[:, varlist_op].copy()

In [11]:
# convert string columns into numeric columns
value_map = {'-1 Inapplicable': -1, ' 1 Yes': 1}
df = r1_op_new.copy()

looplist = ['op1outhlp', 'op1insdhlp', 'op1bedhlp', 'op1launhlp', 'op1shophlp', 'op1mealhlp', 'op1bankhlp', 'op1moneyhlp',
            'op1eathlp', 'op1bathhlp', 'op1toilhlp', 'op1dreshlp', 'op1medshlp', 'op1dochlp', 'op1insurhlp', 
            'op1ishelper']

for col in looplist:
    df[col] = df[col].map(value_map)

In [12]:
# extract days and hours information from categorical columns
extractlist = ['op1numdayswk', 'op1numdaysmn', 'op1numhrsday', 'op1dhrsmth']
for col in extractlist:
    # convert category column to string
    df[col] = df[col].astype(str)
    df[col] = df[col].str.extract(r'([+-]?\d+)')

In [13]:
# create a dummy variable 'got_help' which equals to 1 if sp got helped in any activity or op identified as a helper
df['got_help'] = (df[looplist] == 1).any(axis=1).astype(int)

# create formal_care dummy which equals to 1 if op is a paid helper
df['formal_care'] = np.where(df['op1paidhelpr'] == ' 1 Yes', 1, 0)
# create informal_care dummy which equals to 1 if op is not a paid helper
helplist = [' 2 No', '-8 DK', '-7 RF']
df['informal_care'] = np.where((df['got_help'] == 1) & (df['op1paidhelpr'].isin(helplist)), 1, 0)
df['got_help'] = np.where((df['got_help'] < 1) & (df['formal_care'] ==1 ), 1, df['got_help'])

In [14]:
# a sp may receive care from multiple ops, we consider a 
# need so sum the hours across different op for each sp
# also note that a sp can receive formal and informal care at the same time
hours_data = df[['spid', 'got_help', 'formal_care', 'informal_care', 'op1dhrsmth']].copy()
hours_data = hours_data.astype(int)

formal_data = hours_data[hours_data.formal_care == 1]
informal_data = hours_data[hours_data.informal_care == 1]
nohelp_data = hours_data[hours_data.got_help == 0]

# define functions to apply to columns after groupby
max_col = lambda x: x.max()
sum_pos_col = lambda x: x[x > 0].sum()

formal_hours = formal_data.groupby('spid').agg({'got_help': max_col, 'formal_care': max_col, 'informal_care': max_col,
                                                'op1dhrsmth': sum_pos_col}).reset_index()
informal_hours = informal_data.groupby('spid').agg({'got_help': max_col, 'formal_care': max_col, 'informal_care': max_col,
                                                    'op1dhrsmth': sum_pos_col}).reset_index()
nohelp_data = nohelp_data.groupby('spid').agg({'got_help': max_col, 'formal_care': max_col, 'informal_care': max_col,
                                               'op1dhrsmth': sum_pos_col}).reset_index()

# rename columns for merge
formal_hours = formal_hours.rename(columns={'op1dhrsmth': 'formal_hrs'})
informal_hours = informal_hours.rename(columns={'op1dhrsmth': 'informal_hrs'})

In [15]:
# merge formal_hours and informal_hours, now we know for each sp, how many hours of each kind of care he/she got.
merged = pd.merge(formal_hours, informal_hours, on='spid', how='outer', indicator=True)

# recreate got_help, formal_care and informal_care dummies
merged['got_help'] = np.nanmax(merged[['got_help_x', 'got_help_y']], axis = 1)
merged['formal_care'] = np.nanmax(merged[['formal_care_x', 'formal_care_y']], axis = 1)
merged['informal_care'] = np.nanmax(merged[['informal_care_x', 'informal_care_y']], axis = 1)

# drop redundant columns
merged = merged.drop(['got_help_x', 'got_help_y', 'formal_care_x', 'formal_care_y', 'informal_care_x', 'informal_care_y',
                      '_merge'], axis = 1)

# rename columns in nohelp_data
nohelp_data = nohelp_data.rename(columns={'op1dhrsmth': 'formal_hrs'})
nohelp_data['informal_hrs'] = 0
# stack nohelp_data and merged
sp_hours = pd.concat([merged, nohelp_data], axis=0)
sp_hours = sp_hours.sort_values(by = 'spid').reset_index(drop=True)
sp_hours.fillna(0, inplace=True)

In [16]:
# if hours information is missing, drop that person from the sample
droplist_formal = sp_hours[(sp_hours.formal_hrs==0) & (sp_hours.formal_care==1)].index.to_list()
droplist_informal = sp_hours[(sp_hours.informal_hrs==0) & (sp_hours.informal_care==1)].index.to_list()

sp_hours = sp_hours.drop(droplist_formal+droplist_informal)

In [17]:
# merge care data with sp file
r1_sp_merged = pd.merge(r1_sp, sp_hours, on='spid', how='left')

In [18]:
# extract inforamtion about sp's children
children_op = r1_op_new[['spid', 'opid', 'op1relatnshp', 'op1martlstat', 'op1numchldrn', 'op1numchdu18']].copy()
children_op = children_op[children_op.op1relatnshp.isin([' 3 DAUGHTER', ' 4 SON'])]

# label children who was not married 
marstat = [' 6 NEVER MARRIED', ' 4 DIVORCED', ' 3 SEPARATED', ' 5 WIDOWED']
children_op['not_married_chld'] = np.where(children_op['op1martlstat'].isin(marstat), 1, 0)

# label children who did not have their own children
children_op['no_child_chld'] = np.where(children_op['op1numchldrn']==0, 1, 0)
# label children who did not have children age<18
children_op['no_child18_chld'] = np.where(children_op['op1numchdu18']==0, 1, 0)

# extract information about whether each sp had a child who was not married, who did not have children
# and who did not have children under age 18
df = children_op[['spid', 'not_married_chld', 'no_child_chld', 'no_child18_chld']]
new_df = df.groupby('spid').max().reset_index()

In [19]:
# merge children data with sp file
r1_sp_merged = pd.merge(r1_sp_merged, new_df, on='spid', how='left')

# remove the annoying prefixes in column names
prefixes = ['^r1', '^is1', '^hc1', '^ht1', '^hh1', '^cs1', '^sd1', '^el1', '^rl1', '^ip1', '^hp1', '^ia1']
pattern = '|'.join(prefixes)
r1_sp_merged.columns = r1_sp_merged.columns.str.replace(pattern, '')

<ipython-input-19-23f530c877e7>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  r1_sp_merged.columns = r1_sp_merged.columns.str.replace(pattern, '')


In [20]:
r1_sp_merged.to_csv('r1_data.csv', index = False)

# construct a sample of round 5 sp

In [21]:
# lists of relevant variables
varlist_sp = ['spid', 'r5dresid', 'is5resptype', 'r5dgender', 'r5d2intvrage', 'cog_score', 'cog_score_t1',
              'hc5health', 'hc5disescn3', 'hc5disescn8', 'hc5hosptstay', 'hc5hosovrnht', 'hc5aslep30mn', 'hc5sleepmed',
              'ht5retiresen',
              'hh5martlstat', 'hh5dspageall', 'hh5spouseduc', 'hh5spoupchlp', 'hh5livwthspo', 'hh5dlvngarrg', 'hh5dhshldchd',
              'hh5dlvngarrg',
              'cs5dnumchild', 'cs5dnumdaugh', 'sd5smokedreg', 'sd5smokesnow', 
              'el5hlthchild', 'el5fingrowup', 'el5higstschl', 'rl5spkothlan', 'ip5nginsnurs',
              'hp5ownrentot', 'hp5mrtpadoff', 'ia5totinc', 'ia5toincimf', 'ia5toincim1']
varlist_help = ['mo5douthelp', 'mo5dinsdhelp', 'mo5dbedhelp', 'dm5helpmobil', 'ha5moneyhlp', 'sc5eathlp',
                'sc5bathhlp', 'sc5toilhlp', 'sc5dreshlp', 'mc5medstrk', 'mc5howpkupm3']
varlist_op = ['spid', 'opid', 'op5dgender', 'op5relatnshp', 'op5leveledu', 'op5childinhh', 'op5martlstat', 'op5numchldrn',
              'op5numchdu18',
              'op5ishelper', 'op5helpsched',
              'op5numdayswk', 'op5numdaysmn', 'op5numhrsday', 'op5paidhelpr', 'op5dhrsmth',
              'op5outhlp', 'op5insdhlp', 'op5bedhlp', 'op5launhlp', 'op5shophlp', 'op5mealhlp', 'op5bankhlp', 'op5moneyhlp',
              'op5eathlp', 'op5bathhlp', 'op5toilhlp', 'op5dreshlp', 'op5medshlp', 'op5dochlp', 'op5insurhlp']

In [22]:
r5_sp = r5_data.loc[:, varlist_sp].copy()
r5_op_new = r5_op.loc[:, varlist_op].copy()

In [23]:
# convert string columns into numeric columns
value_map = {'-1 Inapplicable': -1, '1 YES': 1}
df = r5_op_new.copy()

looplist = ['op5outhlp', 'op5insdhlp', 'op5bedhlp', 'op5launhlp', 'op5shophlp', 'op5mealhlp', 'op5bankhlp', 'op5moneyhlp',
            'op5eathlp', 'op5bathhlp', 'op5toilhlp', 'op5dreshlp', 'op5medshlp', 'op5dochlp', 'op5insurhlp', 
            'op5ishelper']

for col in looplist:
    df[col] = df[col].map(value_map)

In [24]:
# fix the coding difference between round 1 and round 5
df['op5dhrsmth'] = np.where(df['op5dhrsmth'] == '9999 Not codeable, <1 hour/day', 'Not codeable, <1 hour/day', df['op5dhrsmth'])

# extract days and hours information from categorical columns
extractlist = ['op5numdayswk', 'op5numdaysmn', 'op5numhrsday', 'op5dhrsmth']
for col in extractlist:
    # convert category column to string
    df[col] = df[col].astype(str)
    df[col] = df[col].str.extract(r'([+-]?\d+)')

In [25]:
# create a dummy variable 'got_help' which equals to 1 if sp got helped in any activity or op identified as a helper
df['got_help'] = (df[looplist] == 1).any(axis=1).astype(float)

# create formal_care dummy which equals to 1 if op is a paid helper
df['formal_care'] = np.where(df['op5paidhelpr'] == ' 1 Yes', 1, 0)
# create informal_care dummy which equals to 1 if op is not a paid helper
helplist = [' 2 No', '-8 DK', '-7 RF']
df['informal_care'] = np.where((df['got_help'] == 1) & (df['op5paidhelpr'].isin(helplist)), 1, 0)
df['got_help'] = np.where((df['got_help'] < 1) & (df['formal_care'] ==1 ), 1, df['got_help'])

In [26]:
# a sp may receive care from multiple ops, we consider a 
# need so sum the hours across different op for each sp
# also note that a sp can receive formal and informal care at the same time
hours_data = df[['spid', 'got_help', 'formal_care', 'informal_care', 'op5dhrsmth']].copy()
hours_data = hours_data.astype(float)

formal_data = hours_data[hours_data.formal_care == 1]
informal_data = hours_data[hours_data.informal_care == 1]
nohelp_data = hours_data[hours_data.got_help == 0]

# define functions to apply to columns after groupby
max_col = lambda x: x.max()
sum_pos_col = lambda x: x[x > 0].sum()

formal_hours = formal_data.groupby('spid').agg({'got_help': max_col, 'formal_care': max_col, 'informal_care': max_col,
                                                'op5dhrsmth': sum_pos_col}).reset_index()
informal_hours = informal_data.groupby('spid').agg({'got_help': max_col, 'formal_care': max_col, 'informal_care': max_col,
                                                    'op5dhrsmth': sum_pos_col}).reset_index()
nohelp_data = nohelp_data.groupby('spid').agg({'got_help': max_col, 'formal_care': max_col, 'informal_care': max_col,
                                               'op5dhrsmth': sum_pos_col}).reset_index()

# rename columns for merge
formal_hours = formal_hours.rename(columns={'op5dhrsmth': 'formal_hrs'})
informal_hours = informal_hours.rename(columns={'op5dhrsmth': 'informal_hrs'})

In [27]:
# merge formal_hours and informal_hours, now we know for each sp, how many hours of each kind of care he/she got.
merged = pd.merge(formal_hours, informal_hours, on='spid', how='outer', indicator=True)

# recreate got_help, formal_care and informal_care dummies
merged['got_help'] = np.nanmax(merged[['got_help_x', 'got_help_y']], axis = 1)
merged['formal_care'] = np.nanmax(merged[['formal_care_x', 'formal_care_y']], axis = 1)
merged['informal_care'] = np.nanmax(merged[['informal_care_x', 'informal_care_y']], axis = 1)

# drop redundant columns
merged = merged.drop(['got_help_x', 'got_help_y', 'formal_care_x', 'formal_care_y', 'informal_care_x', 'informal_care_y',
                      '_merge'], axis = 1)

# rename columns in nohelp_data
nohelp_data = nohelp_data.rename(columns={'op5dhrsmth': 'formal_hrs'})
nohelp_data['informal_hrs'] = 0
# stack nohelp_data and merged
sp_hours = pd.concat([merged, nohelp_data], axis=0)
sp_hours = sp_hours.sort_values(by = 'spid').reset_index(drop=True)
sp_hours.fillna(0, inplace=True)

In [28]:
# if hours information is missing, drop that person from the sample
droplist_formal = sp_hours[(sp_hours.formal_hrs==0) & (sp_hours.formal_care==1)].index.to_list()
droplist_informal = sp_hours[(sp_hours.informal_hrs==0) & (sp_hours.informal_care==1)].index.to_list()

sp_hours = sp_hours.drop(droplist_formal+droplist_informal)

In [29]:
# merge care data with sp file
r5_sp_merged = pd.merge(r5_sp, sp_hours, on='spid', how='left')

In [30]:
# extract inforamtion about sp's children
children_op = r5_op_new[['spid', 'opid', 'op5relatnshp', 'op5martlstat', 'op5numchldrn', 'op5numchdu18']].copy()
children_op = children_op[children_op.op5relatnshp.isin([' 3 DAUGHTER', ' 4 SON'])]

# label children who was not married 
marstat = [' 6 NEVER MARRIED', ' 4 DIVORCED', ' 3 SEPARATED', ' 5 WIDOWED']
children_op['not_married_chld'] = np.where(children_op['op5martlstat'].isin(marstat), 1, 0)

# label children who did not have their own children
children_op['no_child_chld'] = np.where(children_op['op5numchldrn']==0, 1, 0)
# label children who did not have children age<18
children_op['no_child18_chld'] = np.where(children_op['op5numchdu18']==0, 1, 0)

# extract information about whether each sp had a child who was not married, who did not have children
# and who did not have children under age 18
df = children_op[['spid', 'not_married_chld', 'no_child_chld', 'no_child18_chld']]
new_df = df.groupby('spid').max().reset_index()

In [31]:
# merge children data with sp file
r5_sp_merged = pd.merge(r5_sp_merged, new_df, on='spid', how='left')

# remove the annoying prefixes in column names
prefixes = ['^r5', '^is5', '^hc5', '^ht5', '^hh5', '^cs5', '^sd5', '^el5', '^rl5', '^ip5', '^hp5', '^ia5']
pattern = '|'.join(prefixes)
r5_sp_merged.columns = r5_sp_merged.columns.str.replace(pattern, '')

<ipython-input-31-4e25036f564a>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  r5_sp_merged.columns = r5_sp_merged.columns.str.replace(pattern, '')


In [32]:
r5_sp_merged.to_csv('r5_data.csv', index = False)

In [33]:
# find the persons who were just added in round 5
r15_merged = pd.merge(r1_sp_merged['spid'], r5_sp_merged, on='spid', how='right', indicator=True)
r5_new_sp = r15_merged[r15_merged['_merge'] == 'right_only'].drop(columns=['_merge'])

# concatenate round 5 new persons with round 1 persons
r5_new_sp = r5_new_sp.rename(columns = {'dspageall': 'd2spouage'}) # variable for spouse's age has different names in r1 and r5
r15_concat = pd.concat([r1_sp_merged, r5_new_sp], axis = 0)

In [34]:
r15_concat.to_csv('r15_concat.csv', index = False)

# prepare the sample for regression

In [35]:
r15_concat = pd.read_csv('r15_concat.csv')

In [36]:
# define a function to create dummies based on a categorical column
def create_dummies(df, col):
    counts = df[col].value_counts()
    valid_values = counts[counts > 0].index.tolist()
    filtered_df = df[df[col].isin(valid_values)]
    dummy_cols = pd.get_dummies(filtered_df[col])
    df = pd.concat([df, dummy_cols], axis=1)
    return df

In [37]:
# residential care
# merge 2 (SP interview complete) and 2 (SP interview) into one category also made the strings more concise
r15_concat['dresid'] = r15_concat['dresid'].astype('string')
r15_concat['dresid'] = r15_concat['dresid'].str.replace(r'\d+\s', '')
r15_concat['dresid'] = r15_concat['dresid'].str.replace(r'^Residential [Cc]are.*$', 'Residential care', regex = True)
r15_concat['dresid'] = r15_concat['dresid'].str.replace(r'^Nursing home.*$', 'Nursing home')
r15_concat = create_dummies(r15_concat, 'dresid')

<ipython-input-37-611f7741a449>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  r15_concat['dresid'] = r15_concat['dresid'].str.replace(r'\d+\s', '')
<ipython-input-37-611f7741a449>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  r15_concat['dresid'] = r15_concat['dresid'].str.replace(r'^Nursing home.*$', 'Nursing home')


In [38]:
# age category
r15_concat['age_cate'] = r15_concat['d2intvrage'].apply(lambda x: int(re.findall(r'^\d+', x)[0]))

# whether the survey was answered by a proxy person
r15_concat['proxy'] = np.where(r15_concat['resptype'] == '2 PROXY', 1, 0)

# gender
r15_concat['male'] = np.where(r15_concat['dgender'] == '1 MALE', 1, 0)

# spouse age
# remove missings and convert strings to integer
r15_concat = r15_concat[~r15_concat['d2spouage'].isin(['-8 DK', '-8 - DK', '-7 - RF', '-7 RF'])]
r15_concat['d2spouage'] = r15_concat['d2spouage'].str.strip()
r15_concat['spouse_age'] = r15_concat['d2spouage'].apply(lambda x: int(re.findall(r'^-?\d+', x)[0]))

# spouse need help
# remove missings and convert strings to integer
r15_concat = r15_concat[~r15_concat['spoupchlp'].isin(['-8 DK', '-7 RF'])]
r15_concat['spoupchlp'] = r15_concat['spoupchlp'].str.strip()
r15_concat['spou_need_help'] = r15_concat['spoupchlp'].replace({'2 NO': 0, '-1 Inapplicable': -1, '1 YES': 1})

# live with spouse
# remove missings and convert strings to integer
r15_concat['livwthspo'] = r15_concat['livwthspo'].str.strip()
r15_concat['livewth_spou'] = r15_concat['livwthspo'].replace({'2 NO': 0, '-1 Inapplicable': 0, '1 YES': 1})

# live alone
r15_concat['live_alone'] = np.where(r15_concat['dlvngarrg'].isin(['1 Alone ', '1 Alone']), 1, 0)

# finacial condition when growing up
r15_concat = r15_concat[~r15_concat['fingrowup'].isin(['-8 DK', '-7 RF', '-1 Inapplicable'])]
r15_concat['fingrowup'] = r15_concat['fingrowup'].str.strip()
r15_concat['rich_chld'] = r15_concat['fingrowup'].apply(lambda x: 6-int(re.findall(r'^\d+', x)[0]))

In [39]:
# general health condition
# remove missings and convert strings to integer
r15_concat = r15_concat[r15_concat['health'] != '-8 DK']
r15_concat['health'] = r15_concat['health'].str.strip()
r15_concat['health'] = r15_concat['health'].apply(lambda x: 6-int(re.findall(r'^\d+', x)[0]))

# high blood pressure
# remove missings and convert strings to integer
r15_concat = r15_concat[r15_concat['disescn3'] != '-8 DK']
r15_concat['disescn3'] = np.where(r15_concat['disescn3'] == '7 PREVIOUSLY REPORTED', '1 YES', r15_concat['disescn3'])
r15_concat['disescn3'] = r15_concat['disescn3'].str.strip()
r15_concat['high_blood'] = np.where(r15_concat['disescn3'] == '1 YES', 1, 0)

# stroke
# remove missings and convert strings to integer
r15_concat = r15_concat[r15_concat['disescn8'] != '-8 DK']
r15_concat['disescn8'] = r15_concat['disescn8'].str.strip()
r15_concat['stroke'] = np.where(r15_concat['disescn8'] == '1 YES', 1, 0)

# sleep quality
# remove missings and convert strings to integer
r15_concat = r15_concat[r15_concat['aslep30mn'] != '-8 DK']
r15_concat['aslep30mn'] = r15_concat['aslep30mn'].str.strip()
r15_concat['sleep'] = r15_concat['aslep30mn'].apply(lambda x: int(re.findall(r'^\d+', x)[0]))

# smoked regularly
r15_concat = r15_concat[~r15_concat['smokedreg'].isin(['-8 DK', '-7 RF', '-1 Inapplicable'])]
r15_concat['smokedreg'] = r15_concat['smokedreg'].str.strip()
r15_concat['smoke_reg'] = r15_concat['smokedreg'].apply(lambda x: 2-int(re.findall(r'^\d+', x)[0]))

In [40]:
# education
r15_concat = r15_concat[~r15_concat['higstschl'].isin(['-8 DK', '-7 RF'])]
r15_concat['higstschl'] = r15_concat['higstschl'].str.strip()
# create 4 dummies, baseline is high school drop-out, then high school graduate, 
# then some college (include vocational school ect), then college graduate, then above college.
r15_concat['higschgra'] = np.where(r15_concat['higstschl'] == '4 HIGH SCHOOL GRADUATE (HIGH SCHOOL DIPLOMA OR EQUIVALENT)',
                                   1, 0)
r15_concat['somcol'] = np.where(r15_concat['higstschl'].\
                                isin(['5 VOCATIONAL, TECHNICAL, BUSINESS, OR TRADE SCHOOL CERTIFICATE OR DIPLOMA (BEYOND HIGH SCHOOL LEVEL)',\
                                      '6 SOME COLLEGE BUT NO DEGREE',\
                                      "7 ASSOCIATE'S DEGREE"]), 1, 0)
r15_concat['colgra'] = np.where(r15_concat['higstschl'] == "8 BACHELOR'S DEGREE", 1, 0)
r15_concat['colabo'] = np.where(r15_concat['higstschl'] == "9 MASTER'S, PROFESSIONAL, OR DOCTORAL DEGREE", 1, 0)

# bilingual
r15_concat = r15_concat[~r15_concat['spkothlan'].isin(['-8 DK', '-1 Inapplicable'])]
r15_concat['spkothlan'] = r15_concat['spkothlan'].str.strip()
r15_concat['bilingual'] = r15_concat['spkothlan'].apply(lambda x: 2-int(re.findall(r'^\d+', x)[0]))

In [41]:
# income
r15_concat = r15_concat[~r15_concat['toincim1'].isin(['-1 Inapplicable'])]
r15_concat['income'] = pd.to_numeric(r15_concat['toincim1'])

In [42]:
r15_concat.columns

Index(['spid', 'dresid', 'resptype', 'dgender', 'd2intvrage', 'cog_score', 'cog_score_t1', 'health', 'disescn3', 'disescn8', 'hosptstay', 'hosovrnht', 'aslep30mn', 'sleepmed', 'retiresen', 'martlstat', 'd2spouage', 'spouseduc', 'spoupchlp', 'livwthspo', 'dlvngarrg', 'dhshldchd', 'dlvngarrg.1', 'dnumchild', 'dnumdaugh', 'smokedreg', 'smokesnow', 'hlthchild', 'fingrowup', 'higstschl', 'spkothlan', 'nginsnurs', 'ownrentot', 'mrtpadoff', 'totinc', 'toincimf', 'toincim1', 'formal_hrs', 'informal_hrs', 'got_help', 'formal_care', 'informal_care', 'not_married_chld', 'no_child_chld', 'no_child18_chld', 'Community ', 'Nursing home', 'Residential care', 'age_cate', 'proxy', 'male', 'spouse_age', 'spou_need_help', 'livewth_spou', 'live_alone', 'rich_chld', 'high_blood', 'stroke', 'sleep', 'smoke_reg', 'higschgra', 'somcol', 'colgra', 'colabo', 'bilingual', 'income'], dtype='object')

In [43]:
# relevant variable list
var4reg = ['spid', 'cog_score', 'cog_score_t1', 'health', 'dnumchild', 'dnumdaugh', 'formal_hrs', 'informal_hrs', 
           'got_help', 'formal_care', 'informal_care', 'not_married_chld', 'no_child_chld', 'no_child18_chld', 
           'Community ', 'Nursing home', 'age_cate', 'proxy', 'male', 'spouse_age', 'spou_need_help', 'livewth_spou',
           'live_alone', 'rich_chld', 'high_blood', 'stroke', 'sleep', 'smoke_reg', 'higschgra', 'somcol', 'colgra',
           'colabo', 'bilingual', 'income'
           ]

final_sample = r15_concat.loc[:, var4reg]

In [50]:
# fitler out persons whose initial cognitive test scores are 0
final_sample = final_sample[final_sample['cog_score'] != 0]

In [45]:
# calculate the change in cognitive test score
final_sample['cog_diff'] = final_sample['cog_score_t1'] - final_sample['cog_score']
final_sample['cog_diff_perc'] = final_sample['cog_diff'] / final_sample['cog_score'] * 100

In [53]:
final_sample.describe()

,spid,cog_score,cog_score_t1,health,dnumchild,dnumdaugh,formal_hrs,informal_hrs,got_help,formal_care,informal_care,not_married_chld,no_child_chld,no_child18_chld,Community,Nursing home,age_cate,proxy,male,spouse_age,spou_need_help,livewth_spou,live_alone,rich_chld,high_blood,stroke,sleep,smoke_reg,higschgra,somcol,colgra,colabo,bilingual,income,cog_diff,cog_diff_perc
count,5.360000e+03,5360.000000,5360.000000,5360.000000,5360.000000,5360.000000,5305.000000,5305.000000,5305.000000,5305.000000,5305.000000,4295.000000,4295.000000,4295.000000,5360.000000,5360.0,5360.000000,5360.000000,5360.000000,5360.000000,5360.000000,5360.000000,5360.000000,5360.000000,5360.000000,5360.000000,5360.000000,5360.00000,5360.000000,5360.000000,5360.000000,5360.000000,5360.000000,5.360000e+03,5360.000000,5360.000000
mean,1.108402e+07,14.796331,16.709188,3.263806,2.916418,1.467724,3.290858,13.339303,0.329123,0.057304,0.308577,0.622119,0.450058,0.642375,0.962687,0.0,2.855037,0.017537,0.374254,2.574254,-0.455784,0.489179,0.335075,2.590672,0.686194,0.102612,3.469403,0.51250,0.299627,0.271642,0.123134,0.105597,0.142537,5.426765e+04,1.912858,34.364501
std,3.103489e+06,5.674641,8.173339,1.076733,1.927427,1.298112,33.696846,62.875561,0.469939,0.232445,0.461950,0.484914,0.497557,0.479357,0.189546,0.0,1.474189,0.131274,0.483975,3.786188,0.583649,0.499930,0.472061,1.021319,0.464082,0.303480,1.283116,0.49989,0.458137,0.444848,0.328622,0.307350,0.349633,1.907195e+05,8.439319,132.612523
min,1.000000e+07,0.666667,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-26.666667,-100.000000
25%,1.000227e+07,10.666667,14.500000,3.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,2.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,3.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000e+04,-0.500000,-2.702703
50%,1.000468e+07,14.833333,19.250000,3.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.0,3.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,3.000000,1.000000,0.000000,4.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2.900000e+04,3.833333,24.337979
75%,1.000683e+07,19.000000,22.500000,4.000000,4.000000,2.000000,0.000000,2.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,4.000000,0.000000,1.000000,6.000000,0.000000,1.000000,1.000000,3.000000,1.000000,0.000000,5.000000,1.00000,1.000000,1.000000,0.000000,0.000000,0.000000,5.500000e+04,7.333333,61.797753
max,2.000251e+07,29.333333,29.000000,5.000000,14.000000,10.000000,974.000000,1072.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,6.000000,1.000000,1.000000,10.000000,1.000000,1.000000,1.000000,5.000000,1.000000,1.000000,5.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,5.700000e+06,20.500000,2825.000000


In [54]:
final_sample.to_csv('final_sample.csv', index = False)